### Task Oriented Dialogue as Dataflow: Structured Seq2Seq Generation

This notebook walks through the main steps for running all code for our project.

### Setup
setup.sh will install the necessary packages for working with the task_oriented_dialogue_as_dataflow_synthesis dataset.

In [ ]:
!bash setup.sh

get_output.sh runs the task_oriented_dialogue_as_dataflow_synthesis code to get
the source and target data.

In [ ]:
!bash data_preprocessing/getoutput.sh

### Data Preprocessing
All of our data preprocessing occurs in data_processing/process_data_combined.py. During the preprocessing phase, we also compute the retrieval mechanism via SentenceBert. Running process_data_combined.py will process and save our data for future use.

In [ ]:
!python3 data_processing/process_data_combined.py

We load in our original train data and our data with the augmented source and retrieval mechanisms.

In [ ]:
# Original data
data_dir = "./model_input"
train_set_path = os.path.join(data_dir,'train_set.pkl')
val_set_path = os.path.join(data_dir,'val_set.pkl')
train_set = load_dataflow(train_set_path)
val_set = laod_dataflow(val_set_path)

In [ ]:
# Data augmented by retrieval mechanism
train_set_path = os.path.join(data_dir,'train_set_retrieval.pkl')
val_set_path = os.path.join(data_dir,'val_set_retrieval.pkl')
train_set_retrieval = load_dataflow(train_set_path)
val_set_retrieval = laod_dataflow(val_set_path)

### Models 
Here, we define, train, and run decoding on our models as desired.

In [1]:
from train_helper import setup_and_start_training, load_model_from_save 
from decoder import store_model_predictons

# Determine whether we should train or not
train = True

In [ ]:
# All models use the same encoder and Seq2Seq structure
from models import Seq2Seq, Encoder

# Global variables defines hyper parameters
from global_variables import HIDDEN_SIZE, EMBED_SIZE, DEVICE, MODEL_FOLDER

#### Baseline Seq2Seq
As described in the paper, the baseline Seq2Seq is a simple Encoder-Decoder architecture with Bahdanau attention.

In [ ]:
from models import Decoder
encoder = Encoder(len(train_set.src_vocab), EMBED_SIZE, HIDDEN_SIZE) 
decoder = Decoder(len(train_set.trg_vocab), EMBED_SIZE, HIDDEN_SIZE)
seq2seq = Seq2Seq(encoder, decoder).to(DEVICE)

In [ ]:
# Train or load model
model_save_name = 'seq2seq.pt'
if train_model:
    setup_and_start_training(seq2seq, train_set, val_set, model_save_name)
else:
    load_model_from_save(seq2seq, model_save_name)

In [ ]:
# Run decoding on validation set
save_name = 'seq2seq'
store_model_predictions(seq2seq, val_set, save_name)

#### Baseline CopyNet
To implement CopyNet, we created a Decoder with a copy mechanism.

In [ ]:
from models import CopyNetDecoder
encoder = Encoder(len(train_set.src_vocab), EMBED_SIZE, HIDDEN_SIZE) 
decoder = CopyNetDecoder(len(train_set.trg_vocab), EMBED_SIZE, HIDDEN_SIZE)
copynet = Seq2Seq(encoder, decoder).to(DEVICE)

In [ ]:
# Train or load model
model_save_name = 'copynet.pt'
if train_model:
    setup_and_start_training(copynet, train_set, val_set, model_save_name)
else:
    load_model_from_save(copynet, model_save_name)

In [ ]:
# Run decoding on validation set
save_name = 'copynet'
store_model_predictions(copynet, val_set, save_name)

#### CopyNet + Retrieval
Because our data preprocessing pipeline automatically appended retrieved sentences to our source set, we rerun the same copynet model but on the new dataset.

In [ ]:
encoder = Encoder(len(train_set.src_vocab), EMBED_SIZE, HIDDEN_SIZE) 
decoder = CopyNetDecoder(len(train_set.trg_vocab), EMBED_SIZE, HIDDEN_SIZE)
copynetWithRetrieval = Seq2Seq(encoder, decoder).to(DEVICE)

In [ ]:
# Train or load model
model_save_name = 'copynetWithRetrieval.pt'
if train_model:
    setup_and_start_training(copynetWithRetrieval, train_set_retrieval, 
                             val_set_retrieval, model_save_name)
else:
    load_model_from_save(copynetWithRetrieval, model_save_name)

In [ ]:
# Run decoding on validation set
save_name = 'copynetWithRetrieval'
store_model_predictions(copynetWithRetrieval, val_set_retrieval, save_name)

#### CopyNet + Attention
We modified the CopyNet decoder such that it used Bahdanau attention.

In [ ]:
from models import CopyNetDecoderWithAttention
encoder = Encoder(len(train_set.src_vocab), EMBED_SIZE, HIDDEN_SIZE) 
decoder = CopyNetDecoderWithAttention(len(train_set.trg_vocab), EMBED_SIZE, HIDDEN_SIZE)
copynetWithAttention = Seq2Seq(encoder, decoder).to(DEVICE)

In [ ]:
# Train or load model
model_save_name = 'copynetWithAttention.pt'
if train_model:
    setup_and_start_training(copynetWithAttention, train_set, val_set, model_save_name)
else:
    load_model_from_save(copynetWithAttention, model_save_name)

In [ ]:
# Run decoding on validation set
save_name = 'copynetWithAttention'
store_model_predictions(copynetWithAttention, val_set, save_name)

#### CopyNet + Attention + Retrieval
We provide a version of CopyNet with Attention and Retrieval.

In [ ]:
encoder = Encoder(len(train_set.src_vocab), EMBED_SIZE, HIDDEN_SIZE) 
decoder = CopyNetDecoderWithAttention(len(train_set.trg_vocab), EMBED_SIZE, HIDDEN_SIZE)
copynetWithAttentionAndRetrieval = Seq2Seq(encoder, decoder).to(DEVICE)

In [ ]:
# Train or load model
model_save_name = 'copynetWithAttentionAndRetrieval.pt'
if train_model:
    setup_and_start_training(copynetWithAttentionAndRetrieval, train_set_retrieval, 
                             val_set_retrieval, model_save_name)
else:
    load_model_from_save(copynetWithAttentionAndRetrieval, 
                         model_save_name)

In [ ]:
# Run decoding on validation set
save_name = 'copynetWithAttentionAndRetrieval'
store_model_predictions(copynetWithAttentionAndRetrieval, val_set_retrieval, 
                        save_name)

#### Transformer Encoder + CopyNet Decoder
We attempted to incorporate a transformer encoder into the model. This was as follows.

In [ ]:
from train_helper import CustomTransformerEncoder
encoder = CustomTransformerEncoder(EMBED_SIZE, vocab_size=len(train_set.src_vocab)) 
decoder = CopyNetDecoder(len(train_set.trg_vocab), EMBED_SIZE, HIDDEN_SIZE)
transformerCopynet = Seq2Seq(encoder, decoder).to(DEVICE)

In [ ]:
# Train or load model
model_save_name = 'transformerCopynet.pt'
if train_model:
    setup_and_start_training(transformerCopynet, train_set, 
                             val_set, model_save_name)
else:
    load_model_from_save(transformerCopynet, 
                         model_save_name)

In [ ]:
# Run decoding on validation set
save_name = 'transformerCopynet'
store_model_predictions(transformerCopynet, val_set, 
                        save_name)